In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sun Jul 19 10:03:09 2020

To install vpython:
    
pip install vpython

@author: emilio
"""
from vpython import*
import numpy as np
scene = canvas()
# Hard-sphere gas.

# -*- coding: utf-8 -*-
"""
Created on Sun Jul 19 10:03:09 2020

To install vpython:
    
pip install vpython

@author: emilio
"""
from vpython import*
import numpy as np
scene = canvas()
# Hard-sphere gas.

win = 500

Natoms = 200  # change this to have more or fewer atoms
NA=6.022E23
# Typical values
L = 1 # container is a cube L on a side
V = L**3
gray = color.gray(0.7) # color of edges of container
mass = 4E-3/NA # helium mass
Ratom = 0.03 # wildly exaggerated size of helium atom
k = 1.38064852E-23 # Boltzmann constant
T = float(input('Temperature of the simulation(K): '))
dt = 1E-5

animation = canvas( width=win, height=win, align='left')
animation.range = L
animation.title = 'Simulation with '+str(Natoms)+' "hard-sphere" gas particles at T = '+str(T)+' K'
s = """<b>Theoretical and simulated speed distributions.</b>
  Initially all atoms have the same speed, 
  but collisions change the speeds of the 
  colliding atoms. 
  
  One of the atoms is marked and leaves a 
  trail so you can follow its path.
  
  While speeds are realistic, the collision
  frequency and mean free path are not
"""
animation.caption = s

d = L/2+Ratom
r = 0.005
boxbottom = curve(color=gray, radius=r)
boxbottom.append([vector(-d,-d,-d), vector(-d,-d,d), vector(d,-d,d), vector(d,-d,-d), vector(-d,-d,-d)])
boxtop = curve(color=gray, radius=r)
boxtop.append([vector(-d,d,-d), vector(-d,d,d), vector(d,d,d), vector(d,d,-d), vector(-d,d,-d)])
vert1 = curve(color=gray, radius=r)
vert2 = curve(color=gray, radius=r)
vert3 = curve(color=gray, radius=r)
vert4 = curve(color=gray, radius=r)
vert1.append([vector(-d,-d,-d), vector(-d,d,-d)])
vert2.append([vector(-d,-d,d), vector(-d,d,d)])
vert3.append([vector(d,-d,d), vector(d,d,d)])
vert4.append([vector(d,-d,-d), vector(d,d,-d)])

Atoms = []
p = []
apos = []
mfp = [0] * Natoms
pavg = sqrt(2*mass*1.5*k*T) # average kinetic energy p**2/(2mass) = (3/2)kT
    
for i in range(Natoms):
    x = L*random()-L/2
    y = L*random()-L/2
    z = L*random()-L/2
    if i == 0:
        Atoms.append(sphere(pos=vector(x,y,z), radius=Ratom, color=color.cyan, make_trail=True, retain=100, trail_radius=0.3*Ratom))
    else: Atoms.append(sphere(pos=vector(x,y,z), radius=Ratom, color=gray))
    apos.append(vec(x,y,z))
    theta = pi*random()
    phi = 2*pi*random()
    px = pavg*sin(theta)*cos(phi)
    py = pavg*sin(theta)*sin(phi)
    pz = pavg*cos(theta)
    p.append(vector(px,py,pz))

deltav = 100 # binning for v histogram

def barx(v):
    return int(v/deltav) # index into bars array

#nhisto = int(4500/deltav)
nhisto = int(5000/deltav)

histo = []
for i in range(nhisto): histo.append(0.0)
histo[barx(pavg/mass)] = Natoms

xmax = sqrt(2 * k * T / mass) + 2.5 * sqrt(k * T /mass)
ymax = Natoms*4*((mass/(2*pi*k*T))**0.5)/np.e*deltav*1.2
gg = graph( width=win, height=0.4*win, xmax=xmax, align='left',
    xtitle='speed, m/s', ytitle='Number of atoms', ymax=ymax)

theory = gcurve( color=color.blue )
dv = 10
for v in range(0,5001+dv,dv):  # theoretical prediction
    theory.plot( v, (deltav/dv)*Natoms*4*pi*((mass/(2*pi*k*T))**1.5) *exp(-0.5*mass*(v**2)/(k*T))*(v**2)*dv )

accum = []
#for i in range(int(3000/deltav)): accum.append([deltav*(i+.5),0])
for i in range(int(5000/deltav)): accum.append([deltav*(i+.5),0])
vdist = gvbars(color=color.red, delta=deltav )

def interchange(v1, v2):  # remove from v1 bar, add to v2 bar
    barx1 = barx(v1)
    barx2 = barx(v2)
    if barx1 == barx2:  return
    if barx1 >= len(histo) or barx2 >= len(histo): return
    histo[barx1] -= 1
    histo[barx2] += 1
    
def checkCollisions():
    hitlist = []
    r2 = 2*Ratom
    r2 *= r2
#    print('r2: ',r2)
    for i in range(Natoms):
        ai = apos[i]
        for j in range(i) :
            aj = apos[j]
            dr = ai - aj
#            dij = sqrt(dr.x**2+dr.y**2+dr.z**2)
            if mag2(dr) < r2: hitlist.append([i,j])
#            if mag2(dr) < r2: print('dij',dij)
    return hitlist

nhisto = 0 # number of histogram snapshots to average
z = 0
nc = 0
gmfp = [0] * Natoms
cnts = [0] * Natoms
ti = [0] * Natoms
tiref = [0] * Natoms
gti = [0] * Natoms
t = 0
ncoll = 0
#while True:
for nsteps in range(50000):
    rate(300)
    # Accumulate and average histogram snapshots
    for i in range(len(accum)): accum[i][1] = (nhisto*accum[i][1] + histo[i])/(nhisto+1)
    if nhisto % 10 == 0:
        vdist.data = accum
    nhisto += 1

    # Update all positions
    t = t + dt
    for i in range(Natoms):
        aposold = apos[i]
        Atoms[i].pos = apos[i] = apos[i] + (p[i]/mass)*dt
        ti[i] = t - tiref[i]
        # distance travelled by an atom before collision
        dist = ((apos[i].x - aposold.x)**2 + (apos[i].y - aposold.y)**2 + (apos[i].z - aposold.z)**2 )**0.5
        mfp[i] = mfp[i] + dist    
    
    v_square = 0.
    for i in range(Natoms):
        vi = p[i]/mass
        v_square = v_square + vi.x * vi.x + vi.y * vi.y + vi.z * vi.z
#    print(mass * v_square / 3, Natoms * k * T)   
    P = mass * v_square / 3 / V

    
    # Check for collisions
    hitlist = checkCollisions()

    # If any collisions took place, update momenta of the two atoms
    
    
    for ij in hitlist:
        i = ij[0]
        j = ij[1]
        gmfp[i] += mfp[i]
        gmfp[j] += mfp[j]
        gti[i] += ti[i]
        gti[j] += ti[j]
        cnts[i] += 1
        cnts[j] += 1
        mfp[i] = 0
        mfp[j] = 0
        tiref[i] = t
        tiref[j] = t
        ptot = p[i]+p[j]
        posi = apos[i]
        posj = apos[j]
        vi = p[i]/mass
        vj = p[j]/mass
        vrel = vj-vi
        a = vrel.mag2
        if a == 0: continue;  # exactly same velocities
        rrel = posi-posj
        if rrel.mag < Ratom:
#            print(rrel.mag,Ratom)
            continue # one atom went all the way through another
    
        # theta is the angle between vrel and rrel:
        dx = dot(rrel, vrel.hat)       # rrel.mag*cos(theta)
        dy = cross(rrel, vrel.hat).mag # rrel.mag*sin(theta)
        # alpha is the angle of the triangle composed of rrel, path of atom j, and a line
        #   from the center of atom i to the center of atom j where atome j hits atom i:
        value = dy/(2*Ratom)
        if value>1: value =1
        if value<-1: value=-1
        alpha = asin(value) 
        d = (2*Ratom)*cos(alpha)-dx # distance traveled into the atom from first contact
        deltat = d/vrel.mag         # time spent moving from first contact to position inside atom
                
#        z = z + 1 / deltat
        
        posi = posi-vi*deltat # back up to contact configuration
        posj = posj-vj*deltat
#        print('deltat',deltat)
        mtot = 2*mass
        pcmi = p[i]-ptot*mass/mtot # transform momenta to cm frame
        pcmj = p[j]-ptot*mass/mtot
        rrel = norm(rrel)
        pcmi = pcmi-2*pcmi.dot(rrel)*rrel # bounce in cm frame
        pcmj = pcmj-2*pcmj.dot(rrel)*rrel
        p[i] = pcmi+ptot*mass/mtot # transform momenta back to lab frame
        p[j] = pcmj+ptot*mass/mtot
        apos[i] = posi+(p[i]/mass)*deltat # move forward deltat in time
        apos[j] = posj+(p[j]/mass)*deltat
        interchange(vi.mag, p[i].mag/mass)
        interchange(vj.mag, p[j].mag/mass)
    ncoll += len(hitlist)
    sigma = np.pi * (2*Ratom)**2
    gmfp0 = (k* T / P) / (sigma * (2**0.5) )
    MFP = 0
    Zco = 0
    for i in range(Natoms):
        if cnts[i]>0: 
            MFP += gmfp[i]/cnts[i]
            Zco += cnts[i] / gti[i]
#    print(MFP/Natoms,gmfp0,gmfp0/(MFP/Natoms))
    z0 = Natoms / V * sigma * sqrt(2) * sqrt(8 * k * T / np.pi / mass )
    if nsteps == 0: print("    Time    Coll. freq. Mean free path")
    if nsteps % 500 == 0: print("%8.2e %6.0f(%6.0f) %6.3f(%6.3f)" %(t, ncoll/t/Natoms*2, z0, MFP/Natoms,gmfp0))

    for i in range(Natoms):
        loc = apos[i]
        if abs(loc.x) > L/2:
            if loc.x < 0: p[i].x =  abs(p[i].x)
            else: p[i].x =  -abs(p[i].x)
        
        if abs(loc.y) > L/2:
            if loc.y < 0: p[i].y = abs(p[i].y)
            else: p[i].y =  -abs(p[i].y)
        
        if abs(loc.z) > L/2:
            if loc.z < 0: p[i].z =  abs(p[i].z)
            else: p[i].z =  -abs(p[i].z)
    